In [1]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 512 kB 2.9 MB/s eta 0:00:01
     |████████████████████████████████| 39.1 MB 63.1 MB/s eta 0:00:01    |█▎                              | 1.5 MB 63.1 MB/s eta 0:00:01
     |████████████████████████████████| 116 kB 67.5 MB/s eta 0:00:01
     |████████████████████████████████| 143 kB 79.3 MB/s eta 0:00:01
     |████████████████████████████████| 987 kB 42.4 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 726 kB/s  eta 0:00:01
     |████████████████████████████████| 194 kB 98.6 MB/s eta 0:00:01
     |████████████████████████████████| 110 kB 55.6 MB/s eta 0:00:01
     |████████████████████████████████| 289 kB 68.2 MB/s eta 0:00:01
     |████████████████████████████████| 148 kB 20.2 MB/s eta 0:00:01
     |████████████████████████████████| 99 kB 1.1 MB/s  eta 0:00:01
     |████████████████████████████████| 136 kB 60.3 MB/s eta 0:00:01
ERROR: huggingface-hub 0.16.4

In [2]:
import torch
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import BlipProcessor, BlipForQuestionAnswering, Trainer, TrainingArguments
from PIL import Image
import os
import json
from tqdm import tqdm

ModuleNotFoundError: No module named 'torch'

In [ ]:
# from datasets import load_dataset

# dataset = load_dataset("liuhaotian/LLaVA-Instruct-150K", split='train', streaming=True)

In [ ]:
# print(next(iter(dataset)))

In [ ]:
!pip install wget


  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=56fb3c4085d61dbb76589e076d9cd4fe3c541cfa0771eedb6601272a2c558a0a
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
# Constants for directory and model
IMAGE_DIR = '/content/drive/My Drive/coco-dataset/train2014'
MODEL_NAME = 'Salesforce/blip-vqa-base'
ANNOTATIONS_FILE = '/content/drive/My Drive/llava_instruct_150k.json'

In [ ]:
# Load JSON data
with open(ANNOTATIONS_FILE, 'r') as file:
    annotations_json = json.load(file)

In [ ]:
# Load the model and processor
processor = BlipProcessor.from_pretrained(MODEL_NAME)
model = BlipForQuestionAnswering.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

BlipForQuestionAnswering(
  (vision_model): BlipVisionModel(
    (embeddings): BlipVisionEmbeddings(
      (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (encoder): BlipEncoder(
      (layers): ModuleList(
        (0-11): 12 x BlipEncoderLayer(
          (self_attn): BlipAttention(
            (dropout): Dropout(p=0.0, inplace=False)
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (projection): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): BlipMLP(
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
      )
    )
    (post_layernorm): LayerNorm((768,), eps=1e-05, e

In [ ]:
import torch
from PIL import Image
import os
from torchvision import transforms

class VQADataset(torch.utils.data.Dataset):
    """Custom VQA dataset."""

    def __init__(self, annotations, img_dir, processor):
        """
        annotations: List containing all the annotations.
        img_dir: Directory where images are stored.
        processor: BLIP processor for image and text processing.
        """
        self.annotations = annotations
        self.img_dir = img_dir
        self.processor = processor

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):

        question = self.annotations['conversations'][idx][0]['value']  # Assuming first entry is the question
        answer = self.annotations['conversations'][idx][1]['value']    # Assuming second entry is the answer
        image_id = self.annotations['image'][idx]
        image_path = os.path.join(self.img_dir, f"COCO_train2014_{image_id}")

        if os.path.exists(image_path):
            image = Image.open(image_path).convert("RGB")
        else:
            # If image does not exist, use a zero tensor as a placeholder
            print(f"Image not found: {image_path}, using zero tensor.")
            image = torch.zeros((3, 224, 224))  # Assuming size 224x224, modify as needed

        # Process image and question
        encoding = self.processor(image, question, return_tensors="pt", padding="max_length", truncation=True)

        # Process labels (answers)
        labels = self.processor.tokenizer.encode(answer, max_length=8, padding="max_length", return_tensors='pt').squeeze()

        # Remove batch dimension
        encoding = {k: v.squeeze() for k, v in encoding.items()}
        encoding['labels'] = labels

        return encoding



In [ ]:

# Convert list of dictionaries (loaded from JSON) into a dictionary of lists
data = {
    'id': [],
    'image': [],
    'conversations': []
}

for item in annotations_json:
    data['id'].append(item['id'])
    data['image'].append(item['image'])
    data['conversations'].append(item['conversations'])




In [ ]:
# Apply transformation
dataset = VQADataset(data, IMAGE_DIR, processor)


In [ ]:
dataset

In [ ]:
train_dataloader = DataLoader(dataset, batch_size=4, shuffle=False, pin_memory=True)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=4e-5)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9, last_epoch=-1, verbose=False)

num_epochs = 100
patience = 10
min_eval_loss = float("inf")
early_stopping_hook = 0
tracking_information = []
scaler = torch.cuda.amp.GradScaler()

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [47]:
for epoch in range(num_epochs):
    epoch_loss = 0
    model.train()
    for idx, batch in zip(tqdm(range(len(train_dataloader)), desc='Training batch: ...'), train_dataloader):
        input_ids = batch.pop('input_ids').to(device)
        pixel_values = batch.pop('pixel_values').to(device)
        attention_masked = batch.pop('attention_mask').to(device)
        labels = batch.pop('labels').to(device)

        with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
            outputs = model(input_ids=input_ids,
                        pixel_values=pixel_values,
                        # attention_mask=attention_masked,
                        labels=labels)

        loss = outputs.loss
        epoch_loss += loss.item()
        # loss.backward()
        # optimizer.step()
        optimizer.zero_grad()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

    model.eval()
    eval_loss = 0
    for idx, batch in zip(tqdm(range(len(valid_dataloader)), desc='Validating batch: ...'), valid_dataloader):
        input_ids = batch.pop('input_ids').to(device)
        pixel_values = batch.pop('pixel_values').to(device)
        attention_masked = batch.pop('attention_mask').to(device)
        labels = batch.pop('labels').to(device)

        with torch.amp.autocast(device_type='cuda', dtype=torch.float16):
            outputs = model(input_ids=input_ids,
                        pixel_values=pixel_values,
                        attention_mask=attention_masked,
                        labels=labels)

        loss = outputs.loss
        eval_loss += loss.item()

Training batch: ...:   0%|          | 0/1 [00:00<?, ?it/s]

Image not found: /content/drive/My Drive/coco-dataset/train2014/COCO_train2014_000000033471.jpg, using zero tensor.
Image not found: /content/drive/My Drive/coco-dataset/train2014/COCO_train2014_000000052846.jpg, using zero tensor.
Image not found: /content/drive/My Drive/coco-dataset/train2014/COCO_train2014_000000334872.jpg, using zero tensor.


Training batch: ...:   0%|          | 0/1 [00:00<?, ?it/s]


RuntimeError: stack expects each tensor to be equal size, but got [12] at entry 0 and [19] at entry 1